## Tutorial: Using pd_exp to create experiments with Prisoner's Dilemma Tournaments
This notebook walks through two different experimental setups to showcase the code functionality of the `pd_exp` module.

In [1]:
# Local modules to import
import pd_exp
from helper_funcs import partitions
from settings import player_names, stag

# Other Python modules used
import pandas as pd
from pathlib import Path
import time, json

<generator object partitions at 0x7f63690eb830>


`pd_exp` was designed to collect data from multiple groups of round-robin tournaments. To keep vocabulary consistent, let me introduce some project terminology:
- _Supergame_: a single PD instance between two players that consists of one or more rounds of play.
- _Tournament_: a round-robin tournament between a select group of players. This concept can also be referred to as a team.
- _System_: a collection of one or more teams.

In the code below, I setup data collection for two different experiments. In experiment 1, we consider how measured outcomes differ within one team of deterministic players in two different game variations: classic PD and Stag Hunt. In experiment 2, we expand on experiment 1 to consider multiple different systems, where each system is comprised of two teams.

#### __Team Metrics__
To measure team outcomes, we use the following metrics: 1) minimum of normalized player scores, 2) average of normalized player scores, and 3) average normalized CC distribution.

##### __The Minumum (Min Score) and Average (Avg Score) of Normalized Player Scores__
The normalized score for a player, $j$, is computed as 
$$
S_j = \sum_{i\neq j} \frac{\text{Score-per-turn against opponent } i}{\text{Total opponents}} = \frac{1}{\text{Total opponents}} \sum_{i\neq j} \frac{S_{ji}}{T_{ji}},
$$
where $S_{ji}$ is player $j$'s score against player $i$ and $T_{ji}$ is the total turns played between player $j$ and $i$.

The minimum and average values across all normalized player scores are taken as aggregate group measures to reflect group welfare.

##### __The New Average score
The new average score is computed as 

$$
\frac{\sum_{j=1}^N \sum_{i \neq j} {S_{ji}}}{\sum_{j=1}^N \sum_{i \neq j} {T_{ji}} / 2}
$$



##### __The Average Normalized CC Distribution (Avg CC Dist)__
The normalized CC distribution for a player represents the proportion of turns where they and their opponent chose to cooperate (CC) out of the total turns given. The average across each player's normalized CC distribution is used as an aggregate measure for overall group cooperation. The average normalized CC distribution is computed as 

$$
\frac{1}{N} \sum_{j=1}^N \frac{1}{N-1} \sum_{i \neq j} \frac{CC \text{ count against player } i}{\text{total turns against player }i}
$$

where $N$ is the total number of players in a model, $j$ is a given player, and $i$ is every other player.

##### __The New Average Normalized CC Distribution (Avg CC Dist)__
The normalized CC distribution for a player represents the proportion of turns where they and their opponent chose to cooperate (CC) out of the total turns given. The average across each player's normalized CC distribution is used as an aggregate measure for overall group cooperation. The average normalized CC distribution is computed as 

$$
\frac{\sum_{j=1}^N \sum_{i \neq j} {C_{ji}}}{\sum_{j=1}^N \sum_{i \neq j} {T_{ji}}}
$$

where $N$ is the total number of players in a model, $j$ is a given player, and $i$ is every other player.


#### __System Metrics__
To measure system outcomes, we extend the above metrics and use six different measures:
1. System Minimum Score (SYS MIN)
2. System Average Score (SYS AVG)
3. Minimum of Team Average Scores (Min of Team Avgs)
4. Average of Team Minimum Scores (Avg of Team Mins)
5. System CC Distribution Average (SYS CC Dist AVG)
6. System CC Distribution Average (SYS CC Dist MIN)

### Experiment 1: Comparing team performance across classic PD and Stag Hunt

In [2]:
### For experiment 1
# Since this simulation gathers data for systems, I coded systems as ordered-lists of 
# teams. To implement ordered-lists of teams I nested player_names within closed-brackets and then a tuple to create an ordered-list.
# The simulation code allows for processing numerous systems, so I enclose the tournament within another pair
# of closed-brackets and then a tuple.

part_list = tuple([tuple([player_names])]) # 'part' is shorthand for partition

In [3]:
print(part_list)

((['Alternator', 'Anti Tit For Tat', 'Bully', 'Cooperator', 'Cycler DC', 'Defector', 'Suspicious Tit For Tat', 'Tit For Tat', 'Win-Shift Lose-Stay', 'Win-Stay Lose-Shift'],),)


In [4]:
print('Total count of systems: ',len(part_list))  #Should be 5775 for n=12 and k=4
start_time = time.time()

print('Instantiate PD Experiments with different game types')
# To instantiate a PD Experiment, I use the PDExp constructor with at partition list and game-type, which is 
# by default the classic PD)
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)


print('Running experiments and computing data')
# To generate the simulation data and compute the group metrics, I use the run_experiments() method.
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()


print('Saving experiment data')
### For experiment 1
path = 'Data/Experiment1/'
classic_pdExp.save_data(path, 'ClassicPD_test_DELETE')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_test_DELETE')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of systems:  1
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  (['Alternator', 'Anti Tit For Tat', 'Bully', 'Cooperator', 'Cycler DC', 'Defector', 'Suspicious Tit For Tat', 'Tit For Tat', 'Win-Shift Lose-Stay', 'Win-Stay Lose-Shift'],)
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift
Analysing: 100%|██████████| 25/25 [00:00<00:00, 52.01it/s]match length for each user:  [[[3.0, 4.0, 8.0, 21.0, 12.0, 20.0, 30.0, 2.0, 4.0, 6.0], [4.0, 2.0, 10.0, 16.0, 3.0, 6.0, 2.0, 12.0, 14.0, 21.0], [8.0, 10.0, 8.0, 6.0, 4.0, 22.0, 12.0, 9.0, 17.0, 4.0], [21.0, 16.0, 6.0, 3.0, 4.0, 1.0, 3.0, 8.0, 21.0, 6.0], [12.0, 3.0, 4.0, 4.0, 1.0, 2.0, 6.0, 9.0, 12.0, 5.0], [20.0, 6.0, 22.0, 1.0, 2.0, 20.0, 8.0, 8.0, 22.0, 17.0], [30.0, 2.0, 12.0, 3.0, 6.0, 8.0, 8.0, 11.0, 30.0, 1.0], [2.0, 12.0, 9.0, 8.

### Experiment 1: Data Comparison Across Games

In [5]:
# Retrieve saved data
classic_dataf1 = pd.read_csv('Data/Experiment1/ClassicPD_test_DELETE_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf1 = pd.read_csv('Data/Experiment1/StagHunt_test_DELETE_RPST_5_1_0_3.csv', index_col=0)

#### _The System ID_
Since these experiments are with unique players, each system is denoted with a unique ID that is comprised of numbers, commas, and underscores. The numbers represent players and determined by the alphabetical order of the player's strategy out of the total set of player strategies in system. The commas delimit players of the same team, and underscores delimit different teams of the same system. Note the system IDs and team compositions below and how they differ between experiment 1 and experiment 2. 

In [6]:
# I use option_context to customize the limits of the dataframe display
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist
1,"1,2,3,4,5,6,7,8,9,10","Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.284385,1.409722,0.248897


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist
1,"1,2,3,4,5,6,7,8,9,10","Alternator,Anti Tit For Tat,Bully,Cooperator,Cycler DC,Defector,Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.257786,1.994048,0.248897


In [8]:
# SIDENOTE: Documentation (i.e. docstring) for code can be retrieved with '?'. Here are some examples.

?pd_exp

Type:        module
String form: <module 'pd_exp' from '/home/s/PrisonersDilemma-main/pd_exp.py'>
File:        ~/PrisonersDilemma-main/pd_exp.py
Docstring:  
Pd_exp: Prisoner's Dilemma Experimentation

Generate data for numerous Prisoner's Dilemma tournaments or systems of 
tournaments.

Classes:

    PdTournament
        A class to represent a tournament.
    PdSystem 
        Generate data for multiple tournaments and organize it into a dataframe
    PdExp
        Generate data for multiple systems
        
Functions:

    grouper(iterable, n, fillvalue=None) -> iterable of n-sized-chunks
        Collect data into fixed-length chunks or blocks
    avg_normalised_state(object, tuple) -> float
        Returns the tournament average for given state distribution (e.g.
        (C,C), (D,D), (C,D), (D,C))


In [9]:
?pd_exp.PdExp

Init signature: pd_exp.PdExp(tuple_of_systems, game_type=None)
Docstring:     
A class to represent a series of systems. 

...

Attributes
----------
sys_tuple : tuple
    three-dimensional ordered list, or tuple, where the first-dimension is
    a list of systems, the 2nd-dimension is the team list, and the 
    3rd-dimension is the individual players.
game : axelrod.game (object)
    container for game matrix and scoring logic 

Methods
-------
run_experiments:
    Iterates through each system, runs the different tournaments and 
    then saves the data to the data attribute.
save_data(path_to_directory, descrip_name):
    Saves experiment data as a csv file
Init docstring:
Constructs all the necessary attributes for pd experiment object

Parameters
----------
tuple_of_systems : tuple
    a multi-dimensional matrix where each item of the first-dimension is an
    ordered list of systems, and the second dimension is an ordered-list of 
    teams
game_type : axelrod.game (object)
    c

### Experiment 2: Comparing systems of 3 teams across classic PD and Stag Hunt

In [10]:
### For experiment 2
# To create numerous systems with a partitioned list of teams can take time, so I save the tuple of systems 
# to file immediately after computing all k-partition subsets of a set of players  to avoid having to recompute 
# them at each run.

# Checking for txt file with list of systems
p = Path('partition_list_teams_of5_DEMO.txt')
if p.exists():
   with open(p, "r") as f:
       part_list = json.load(f)
else:
   print('creating partitions and saving to file')
   part_list = list(partitions(player_names,5))
   p.touch()
   with open(p, "w") as f:
       json.dump(part_list, f)
print(part_list)

', 'Bully', 'Cycler DC', 'Win-Stay Lose-Shift']], [['Cycler DC', 'Cooperator', 'Defector', 'Alternator', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Anti Tit For Tat', 'Win-Stay Lose-Shift']], [['Cycler DC', 'Cooperator', 'Anti Tit For Tat', 'Defector', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Win-Stay Lose-Shift', 'Alternator']], [['Cycler DC', 'Cooperator', 'Anti Tit For Tat', 'Defector', 'Alternator'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat']], [['Alternator', 'Cooperator', 'Defector', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cycler DC', 'Anti Tit For Tat']], [['Cooperator', 'Anti Tit For Tat', 'Defector', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cycler DC', 'Alternator']], [['Alternator', 'Cooperator', 'Anti Tit For Tat', 'Defector', 'Win-Stay Los

In [11]:
part_list = part_list[:3] # Let's just consider 3 different systems
print('Total count of systems: ',len(part_list))
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)

print('Running experiments and computing data')
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of systems:  3
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [['Alternator', 'Cycler DC', 'Anti Tit For Tat', 'Win-Stay Lose-Shift', 'Suspicious Tit For Tat'], ['Win-Shift Lose-Stay', 'Tit For Tat', 'Bully', 'Cooperator', 'Defector']]
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Cycler DC,Suspicious Tit For Tat,Win-Stay Lose-Shift
Analysing: 100%|██████████| 25/25 [00:00<00:00, 48.22it/s]match length for each user:  [[[3.0, 4.0, 8.0, 21.0, 12.0], [4.0, 20.0, 30.0, 2.0, 4.0], [8.0, 30.0, 6.0, 2.0, 10.0], [21.0, 2.0, 2.0, 16.0, 3.0], [12.0, 4.0, 10.0, 3.0, 6.0]]]
{}
(C, C)
{(C, D): 2, (D, C): 2}
(C, C)
{(C, C): 4, (D, D): 4}
4
{(C, C): 6, (C, D): 5, (D, C): 5, (D, D): 5}
10
{(C, D): 6, (D, C): 6}
(C, C)
{(C, D): 2, (D, C): 2}
(C, C)
{}
(C, C)
{(C, C): 15, (D, C): 1, (D, D): 14}
25
{(C, D): 1, (D, C): 1}
(C, C)
{(C, D): 2, (D, C): 1, (D, D): 1}
(C, C)
{(C, C): 4, (D, D): 4}
29
{(

### Experiment 2: Data Comparison Across Systems, Teams, and Games

In [12]:
# Let's compare the results
# Retrieve the stored data
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf = pd.read_csv('Data/Experiment2/StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3.csv', index_col=0)

# Reset the index to accurately count the total systems
classic_dataf = classic_dataf.reset_index(drop=True)
stagHunt_dataf = stagHunt_dataf.reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Experiment2/ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5.csv'

In [ ]:
# The computed results create a large spreadsheet or dataframe. Here is a list of just the column names.
classic_dataf.columns

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team2']])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team2']])

#### How do the systems compare within the same game?

In [ ]:

classic_dataf.loc[:,:'SYS New CC Dist MIN']

In [ ]:

stagHunt_dataf.loc[:,:'SYS New CC Dist MIN']

#### How did team 1 and 2 performance differ across systems?

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2','Team2 Avg Score', 'Team2 New Avg Score', 
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist']])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
                       'display.max_colwidth', None):  # more options can be 
                                                       # specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2','Team2 Avg Score', 'Team2 New Avg Score', 
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist']])

### Experiment 3: A system with 3-player teams

In [26]:
# Checking for txt file with list of systems
# print(player_names)
player_names = ['Cooperator', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Win-Shift Lose-Stay']
p = Path('partition_list_teams_of2_ONLY3PLAYERS_DEMO.txt')
if p.exists():
   with open(p, "r") as f:
       part_list = json.load(f)
else:
   print('creating partitions and saving to file')
   part_list = list(partitions(player_names,3))
   p.touch()
   with open(p, "w") as f:
       json.dump(part_list, f)
print(len(part_list))

10


In [16]:
# part_list = [[('Cooperator', 'Cooperator'), ('Cooperator', 'Defector'), ('Defector', 'Defector')], [('Cooperator', 'Defector'), ('Cooperator', 'Defector'), ('Cooperator', 'Defector')], []]

In [27]:
# part_list = part_list[:3] # Let's just consider 3 different teams
# part_list = [[('Cooperator', 'Defector'), ('Cooperator', 'Cooperator'), ('Defector', 'Defector')]]
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)
# stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)

print('Running experiments and computing data')
classic_pdExp.run_experiments()
# stag_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_2x3_ALL_C_ALL_D')  # CHANGE file_name string when necessary
# stag_pdExp.save_data(path, 'StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  10
[[['Win-Stay Lose-Shift', 'Cooperator', 'Tit For Tat'], ['Suspicious Tit For Tat', 'Win-Shift Lose-Stay', 'Defector']], [['Win-Stay Lose-Shift', 'Defector', 'Cooperator'], ['Suspicious Tit For Tat', 'Win-Shift Lose-Stay', 'Tit For Tat']], [['Defector', 'Cooperator', 'Tit For Tat'], ['Win-Stay Lose-Shift', 'Win-Shift Lose-Stay', 'Suspicious Tit For Tat']], [['Win-Stay Lose-Shift', 'Defector', 'Tit For Tat'], ['Suspicious Tit For Tat', 'Win-Shift Lose-Stay', 'Cooperator']], [['Win-Stay Lose-Shift', 'Win-Shift Lose-Stay', 'Cooperator'], ['Suspicious Tit For Tat', 'Defector', 'Tit For Tat']], [['Win-Shift Lose-Stay', 'Cooperator', 'Tit For Tat'], ['Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat']], [['Win-Stay Lose-Shift', 'Win-Shift Lose-Stay', 'Tit For Tat'], ['Suspicious Tit For Tat', 'Defector', 'Cooperator']], [['Win-Shift Lose-Stay', 'Cooperator', 'Defector'], ['Win-Stay Lose-Shift', 'Suspicious Tit For Tat', 'Tit For Tat']], [['Win-Stay Lose-Shif

### Experiment 3: Comparing the results

In [29]:
# Let's compare the results
# Retrieve the stored data
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_2x3_ALL_C_ALL_D_RPST_3_1_0_5.csv', index_col=0)

# Reset the index to accurately count the total systems
classic_dataf = classic_dataf.reset_index(drop=True)

In [30]:
# The computed results create a large spreadsheet or dataframe. Here is a list of just the column names.
classic_dataf.columns

Index(['System ID', 'SYS MIN Score', 'SYS AVG Score', 'MIN of Team Avgs',
       'AVG of Team Mins', 'SYS CC Dist AVG', 'SYS CC Dist MIN',
       'SYS New CC Dist AVG', 'SYS New CC Dist MIN', 'Team1', 'Player1',
       'P1_Norm_Score', 'Player2', 'P2_Norm_Score', 'Player3', 'P3_Norm_Score',
       'Team1 Avg Score', 'Team1 New Avg Score', 'Team1 Min Score',
       'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2', 'Player1.1',
       'P1_Norm_Score.1', 'Player2.1', 'P2_Norm_Score.1', 'Player3.1',
       'P3_Norm_Score.1', 'Team2 Avg Score', 'Team2 New Avg Score',
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist'],
      dtype='object')

In [31]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team2']])

,System ID,Team1,Team2
0,"4,8,10_6,7,9","Cooperator,Tit For Tat,Win-Stay Lose-Shift","Defector,Suspicious Tit For Tat,Win-Shift Lose-Stay"
1,"4,6,10_7,8,9","Cooperator,Defector,Win-Stay Lose-Shift","Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay"
2,"4,6,8_7,9,10","Cooperator,Defector,Tit For Tat","Suspicious Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift"
3,"6,8,10_4,7,9","Defector,Tit For Tat,Win-Stay Lose-Shift","Cooperator,Suspicious Tit For Tat,Win-Shift Lose-Stay"
4,"4,9,10_6,7,8","Cooperator,Win-Shift Lose-Stay,Win-Stay Lose-Shift","Defector,Suspicious Tit For Tat,Tit For Tat"
5,"4,8,9_6,7,10","Cooperator,Tit For Tat,Win-Shift Lose-Stay","Defector,Suspicious Tit For Tat,Win-Stay Lose-Shift"
6,"8,9,10_4,6,7","Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift","Cooperator,Defector,Suspicious Tit For Tat"
7,"4,6,9_7,8,10","Cooperator,Defector,Win-Shift Lose-Stay","Suspicious Tit For Tat,Tit For Tat,Win-Stay Lose-Shift"
8,"6,9,10_4,7,8","Defector,Win-Shift Lose-Stay,Win-Stay Lose-Shift","Cooperator,Suspicious Tit For Tat,Tit For Tat"
9,"6,8,9_4,7,10","Defector,Tit For Tat,Win-Shift Lose-Stay","Cooperator,Suspicious Tit For Tat,Win-Stay Lose-Shift"


In [32]:

classic_dataf.loc[:,:'SYS New CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN,SYS New CC Dist AVG,SYS New CC Dist MIN
0,"4,8,10_6,7,9",1.000000,2.000000,1.000000,2.000000,0.500000,0.000000,0.260870,0.000000
1,"4,6,10_7,8,9",1.500000,2.236111,2.055556,1.625000,0.222222,0.111111,0.130435,0.086957
2,"4,6,8_7,9,10",1.125000,2.052083,1.875000,1.312500,0.166667,0.000000,0.130435,0.000000
3,"6,8,10_4,7,9",1.750000,2.093750,1.958333,1.906250,0.395833,0.333333,0.228261,0.173913
4,"4,9,10_6,7,8",1.166667,2.145833,1.541667,1.708333,0.250000,0.000000,0.152174,0.000000
5,"4,8,9_6,7,10",1.312500,2.208333,1.625000,1.781250,0.291667,0.000000,0.173913,0.000000
6,"8,9,10_4,6,7",1.312500,2.434028,2.145833,1.635417,0.368056,0.291667,0.206522,0.152174
7,"4,6,9_7,8,10",0.750000,2.312500,2.083333,1.500000,0.250000,0.166667,0.108696,0.043478
8,"6,9,10_4,7,8",1.125000,2.270833,1.750000,1.875000,0.291667,0.000000,0.163043,0.000000
9,"6,8,9_4,7,10",1.166667,2.138889,1.583333,1.708333,0.361111,0.083333,0.217391,0.021739


In [33]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2','Team2 Avg Score', 'Team2 New Avg Score', 
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 New Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team1 New Avg CC Dist,Team2,Team2 Avg Score,Team2 New Avg Score,Team2 Min Score,Team2 Avg CC Dist,Team2 New Avg CC Dist
0,"4,8,10_6,7,9","Cooperator,Tit For Tat,Win-Stay Lose-Shift",3.000000,3.130435,3.000000,1.000000,0.521739,"Defector,Suspicious Tit For Tat,Win-Shift Lose-Stay",1.000000,1.043478,1.000000,0.000000,0.000000
1,"4,6,10_7,8,9","Cooperator,Defector,Win-Stay Lose-Shift",2.416667,2.652174,1.500000,0.333333,0.173913,"Suspicious Tit For Tat,Tit For Tat,Win-Shift Lose-Stay",2.055556,2.434783,1.750000,0.111111,0.086957
2,"4,6,8_7,9,10","Cooperator,Defector,Tit For Tat",2.229167,2.413043,1.500000,0.333333,0.260870,"Suspicious Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",1.875000,1.695652,1.125000,0.000000,0.000000
3,"6,8,10_4,7,9","Defector,Tit For Tat,Win-Stay Lose-Shift",1.958333,1.934783,1.750000,0.333333,0.173913,"Cooperator,Suspicious Tit For Tat,Win-Shift Lose-Stay",2.229167,2.630435,2.062500,0.458333,0.282609
4,"4,9,10_6,7,8","Cooperator,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.750000,2.913043,2.250000,0.500000,0.304348,"Defector,Suspicious Tit For Tat,Tit For Tat",1.541667,1.630435,1.166667,0.000000,0.000000
5,"4,8,9_6,7,10","Cooperator,Tit For Tat,Win-Shift Lose-Stay",2.791667,2.956522,2.250000,0.583333,0.347826,"Defector,Suspicious Tit For Tat,Win-Stay Lose-Shift",1.625000,1.565217,1.312500,0.000000,0.000000
6,"8,9,10_4,6,7","Tit For Tat,Win-Shift Lose-Stay,Win-Stay Lose-Shift",2.722222,2.869565,1.958333,0.444444,0.260870,"Cooperator,Defector,Suspicious Tit For Tat",2.145833,2.500000,1.312500,0.291667,0.152174
7,"4,6,9_7,8,10","Cooperator,Defector,Win-Shift Lose-Stay",2.083333,2.130435,0.750000,0.166667,0.043478,"Suspicious Tit For Tat,Tit For Tat,Win-Stay Lose-Shift",2.541667,2.717391,2.250000,0.333333,0.173913
8,"6,9,10_4,7,8","Defector,Win-Shift Lose-Stay,Win-Stay Lose-Shift",1.750000,1.565217,1.125000,0.000000,0.000000,"Cooperator,Suspicious Tit For Tat,Tit For Tat",2.791667,2.934783,2.625000,0.583333,0.326087
9,"6,8,9_4,7,10","Defector,Tit For Tat,Win-Shift Lose-Stay",1.583333,1.391304,1.166667,0.083333,0.021739,"Cooperator,Suspicious Tit For Tat,Win-Stay Lose-Shift",2.694444,2.956522,2.250000,0.638889,0.413043


### Experiment 4: A system with 2-player teams

In [34]:
# Checking for txt file with list of systems
# print(player_names)
player_names = ['Cooperator', 'Tit For Tat', 'Win-Stay Lose-Shift', 'Defector', 'Suspicious Tit For Tat', 'Win-Shift Lose-Stay']
p = Path('partition_list_teams_of2_ONLY3PLAYERS_DEMO.txt')
if p.exists():
   with open(p, "r") as f:
       part_list = json.load(f)
else:
   print('creating partitions and saving to file')
   part_list = list(partitions(player_names,2))
   p.touch()
   with open(p, "w") as f:
       json.dump(part_list, f)
print(len(part_list))

creating partitions and saving to file
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
sdfgfg
15


In [35]:
# part_list = part_list[:3] # Let's just consider 3 different teams
# part_list = [[('Cooperator', 'Defector'), ('Cooperator', 'Cooperator'), ('Defector', 'Defector')]]
print('Total count of teams: ',len(part_list))
print(part_list)
start_time = time.time()

print('Instantiate PD Experiments with different game types')
classic_pdExp = pd_exp.PdExp(part_list)
# stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)

print('Running experiments and computing data')
classic_pdExp.run_experiments()
# stag_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_3x2_ALL_C_ALL_D')  # CHANGE file_name string when necessary
# stag_pdExp.save_data(path, 'StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of teams:  15
[[('Cooperator', 'Tit For Tat'), ('Suspicious Tit For Tat', 'Defector'), ('Win-Stay Lose-Shift', 'Win-Shift Lose-Stay')], [('Suspicious Tit For Tat', 'Tit For Tat'), ('Defector', 'Cooperator'), ('Win-Stay Lose-Shift', 'Win-Shift Lose-Stay')], [('Suspicious Tit For Tat', 'Cooperator'), ('Defector', 'Tit For Tat'), ('Win-Stay Lose-Shift', 'Win-Shift Lose-Stay')], [('Win-Shift Lose-Stay', 'Tit For Tat'), ('Suspicious Tit For Tat', 'Cooperator'), ('Win-Stay Lose-Shift', 'Defector')], [('Suspicious Tit For Tat', 'Tit For Tat'), ('Win-Shift Lose-Stay', 'Cooperator'), ('Win-Stay Lose-Shift', 'Defector')], [('Suspicious Tit For Tat', 'Win-Shift Lose-Stay'), ('Cooperator', 'Tit For Tat'), ('Win-Stay Lose-Shift', 'Defector')], [('Defector', 'Cooperator'), ('Suspicious Tit For Tat', 'Win-Shift Lose-Stay'), ('Win-Stay Lose-Shift', 'Tit For Tat')], [('Win-Shift Lose-Stay', 'Defector'), ('Suspicious Tit For Tat', 'Cooperator'), ('Win-Stay Lose-Shift', 'Tit For Tat')], [('Wi

### Experiment 4: Comparing the results

In [ ]:
# Let's compare the results
# Retrieve the stored data
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5_RPST_3_1_0_5.csv', index_col=0)

# Reset the index to accurately count the total systems
classic_dataf = classic_dataf.reset_index(drop=True)

In [ ]:
# The computed results create a large spreadsheet or dataframe. Here is a list of just the column names.
classic_dataf.columns

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team2']])

In [ ]:

classic_dataf.loc[:,:'SYS New CC Dist MIN']

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score', 'Team1 New Avg Score', 
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team1 New Avg CC Dist', 'Team2','Team2 Avg Score', 'Team2 New Avg Score', 
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team2 New Avg CC Dist']])